# Using ```.grouby()``` and ```.sum()``` to Aggregate Data
### Finding the number of children in all households for each county in the US

### Import statements

In [2]:
import pandas as pd
import geopandas as gpd

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Reading zip code shapefile

This zipcode file we found an online zip code database, found [here](https://www.unitedstateszipcodes.org/zip-code-database/). This file provides a list of zipcodes as well as the associated county each zip code is in. This is important because we ultimately want a dataframe that contains children household counts by county, but our InfoUSA data only classifies each observation by county FIPS- we will use the database to match each zip code to a county name (Harris County instead of 201). In this dataframe, a lot of the columns are unnecessary; what we will be using and focusing on are the zip code and county columns.

In [3]:
df_zipcodes = pd.read_csv('/hpc/group/codeplus22-vis/us_zipcode_shp_files/zip_code_database.csv')
df_zipcodes.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


### Reading county-level shapefile

We want to create a map of the US using GeoViews, where each county is colored by the number of children in that county. For this, we need a dataframe with geometries for all counties in the US- which we took from the United States Census Bureau's Cartographic Boundary Files (available [here](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)).

In [28]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/county_shp_files/us_counties.shp')
df_counties
df_counties.rename(columns = {'ZCTA5CE10' : 'zip'}, inplace = True)
# df_shp_file = df_shp_file.astype({"zip": int})
# df_shp_file

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


### Merging zipdcode file and county-level shapefile by ```zip```

We are now joining the ```df_zipcodes``` with ```df_shp_file``` by the ```zip``` column, as specified in the ```on``` parameter. This will give us a dataframe with each zip code, the county it is in, and the geometry for that county. We then drop all irrelevant columns. 

In [18]:
df_zip_shp = df_shp_file.merge(df_zipcodes, how = 'left')

df_zip_shp = df_zip_shp[['zip', 'county', 'state', 'geometry']]
df_zip_shp

,zip,county,state,geometry
0,36083,Macon County,AL,"MULTIPOLYGON (((-85.63225 32.28098, -85.62439 ..."
1,35441,Hale County,AL,"MULTIPOLYGON (((-87.83287 32.84437, -87.83184 ..."
2,35051,Shelby County,AL,"POLYGON ((-86.74384 33.25002, -86.73802 33.251..."
3,35121,Blount County,AL,"POLYGON ((-86.58527 33.94743, -86.58033 33.948..."
4,35058,Cullman County,AL,"MULTIPOLYGON (((-86.87884 34.21196, -86.87649 ..."
...,...,...,...,...
33139,10983,Rockland County,NY,"POLYGON ((-73.96564 41.02787, -73.96612 41.029..."
33140,50460,Mitchell County,IA,"POLYGON ((-92.80629 43.23026, -92.80354 43.232..."
33141,40870,Harlan County,KY,"POLYGON ((-83.19264 36.91650, -83.19086 36.916..."
33142,40914,Clay County,KY,"POLYGON ((-83.62748 37.07419, -83.62455 37.073..."


### Reading InfoUSA data

This pre-processed InfoUSA data provides the children counts of households as well as the zip code and county FIPS of the household. This file also includes the transformed latitude and longitude coordinates. We will then get rid of certain columns that are unnecessary because we will be merging the dataframes by ```zip``` column.

In [31]:
test = pd.read_csv('/hpc/group/codeplus22-vis/infousa/derived set/2020/Household_Ethnicity_zip_49320_year_2020.txt', sep = '\t')

Index(['FamilyID', 'location_type', 'primary_family_ind', 'HouseholdStatus',
       'tradeline_count', 'head_hh_age_code', 'length_of_residence',
       'ChildrenHHCount', 'children_ind', 'AddressType', 'mailability_score',
       'wealth_finder_score', 'find_div_1000', 'owner_renter_status',
       'estmtd_home_val_div_1000', 'marital_status', 'ppi_div_1000',
       'CBSACode', 'CBSAType', 'CSACode', 'LOCATIONID', 'HOUSE_NUM',
       'HOUSE_NUM_FRACTION', 'STREET_PRE_DIR', 'STREET_NAME',
       'STREET_POST_DIR', 'STREET_SUFFIX', 'UNIT_TYPE', 'UNIT_NUM', 'BOX_TYPE',
       'BOX_NUM', 'ROUTE_TYPE', 'ROUTE_NUM', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'dpbc', 'vacant', 'USPSNOSTATS', 'census_state_2010',
       'census_county_2010', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010',
       'GE_CENSUS_LEVEL_2010', 'middle_name', 'last_name_1',
       'Ethnicity_Code_1', 'first_name_2', 'last_name_2', 'Ethnicity_Code_2',
       'first_name_3', 'last_name_3', 'Ethnicity_Code_3'],
      dtype='object'

In [34]:
test2 = test[['STATE', 'ZIP', 'census_county_2010', 'census_state_2010', 'dpbc']]
test2

,STATE,ZIP,census_county_2010,census_state_2010,dpbc
0,MI,49320,107,26,223.0
1,MI,49320,107,26,281.0
2,MI,49320,107,26,0.0
3,MI,49320,107,26,876.0
4,MI,49320,107,26,0.0
...,...,...,...,...,...
314,MI,49320,107,26,495.0
315,MI,49320,107,26,123.0
316,MI,49320,107,26,197.0
317,MI,49320,107,26,657.0


In [6]:
df_hh = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_99_final.parquet')
df_hh

,zip,county,state,child_num,has_child,age_code,lat_h_4326,lon_h_4326,lat_h_3857,lon_h_3857
0,18833,113,PA,0,0,K,41.546738,-76.540436,-8.520442e+06,5.093323e+06
1,18833,15,PA,0,0,H,41.590800,-76.424200,-8.507503e+06,5.099879e+06
2,18833,15,PA,1,1,C,41.600392,-76.441724,-8.509454e+06,5.101307e+06
3,18833,15,PA,0,0,L,41.592483,-76.437832,-8.509021e+06,5.100129e+06
4,18833,15,PA,1,1,H,41.566196,-76.347977,-8.499018e+06,5.096218e+06
...,...,...,...,...,...,...,...,...,...,...
190987608,92003,73,CA,0,0,C,33.285885,-117.240445,-1.305115e+07,3.933312e+06
190987609,92003,73,CA,0,0,E,33.284700,-117.210800,-1.304785e+07,3.933154e+06
190987610,92003,73,CA,0,0,G,33.282869,-117.183963,-1.304486e+07,3.932911e+06
190987611,92003,73,CA,0,0,H,33.278284,-117.181181,-1.304455e+07,3.932300e+06


In [7]:
df_hh = df_hh[['zip', 'county', 'state', 'child_num']]
df_hh

,zip,county,state,child_num
0,18833,113,PA,0
1,18833,15,PA,0
2,18833,15,PA,1
3,18833,15,PA,0
4,18833,15,PA,1
...,...,...,...,...
190987608,92003,73,CA,0
190987609,92003,73,CA,0
190987610,92003,73,CA,0
190987611,92003,73,CA,0


### Merge with infousa data with zip and county data

Now, we are merging the ```df_hh```, our household data, with ```df_zip_shp``` so that the resulting dataframe has the zip code, county name, state, of each household along with the child count and geometries.

This ```.merge()``` function will merge the two specified dataframes on the column that is specified in the ```on``` parameter. In this example, we are merging the two dataframes on the ```zip``` column. This merge causes the column in the left dataframe to be called ```county_x``` and the column in the right dataframe ```county_y```, because there were two columns with divergent values named ```county``` in each dataframe. When merging, pandas adds ```_x``` and ```_y``` to differentiate the values from the two while not losing any values. We would only like to keep ```county_y```, because it is the names of each county instead of county FIPS.

In [24]:
df_merged = df_hh.merge(df_zip_shp, on = 'zip')
df_merged

,zip,county_x,state_x,child_num,county_y,state_y,geometry
0,18833,113,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
1,18833,15,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
2,18833,15,PA,1,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
3,18833,15,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
4,18833,15,PA,1,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
...,...,...,...,...,...,...,...
188975504,92003,73,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975505,92003,73,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975506,92003,73,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975507,92003,73,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."


In [21]:
df_merged = df_merged.drop(['county_x'], axis = 1)
df_merged

,zip,state_x,child_num,county_y,state_y,geometry
0,18833,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
1,18833,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
2,18833,PA,1,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
3,18833,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
4,18833,PA,1,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
...,...,...,...,...,...,...
188975504,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975505,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975506,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975507,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."


### Renaming columns

This is an example of code to rename a column name. The first parameter is the original column name, and the second is new name. By setting the ```inplace``` parameter to ```True```, the dataframe is mutated in place (meaning the original copy of the dataframe is mutated).

In [22]:
df_merged.rename(columns = {'county_y' : 'county'}, inplace = True)
df_merged

,zip,state_x,child_num,county,state_y,geometry
0,18833,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
1,18833,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
2,18833,PA,1,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
3,18833,PA,0,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
4,18833,PA,1,Bradford County,PA,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
...,...,...,...,...,...,...
188975504,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975505,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975506,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."
188975507,92003,CA,0,San Diego County,CA,"POLYGON ((-117.26225 33.28622, -117.25940 33.2..."


### Using ```.groupby()``` and ```.sum()``` to find child counts per county

The ```.groupby()``` method takes in the dataframe you specify and groups all of its observations according to the column names you pass into it as parameter. In this case, we are grouping ```df_merged``` by ```county``` to group all observations by county. We are then performing ```.sum()``` on the ```child_num``` column, which returns the sum of all the children across all zip codes in each county. The ```.reset_index()``` method will rearrange the index of the rows in the dataframe so that they start at zero and increase by one for each row, as the indices were shuffled when using the ```.groupby()``` method.

In [23]:
df_child_count = df_merged.groupby('county')['child_num'].sum().reset_index()
df_child_count

,county,child_num
0,Abbeville County,5203
1,Acadia Parish,19191
2,Accomack County,6825
3,Ada County,124318
4,Adair County,15547
...,...,...
1832,Yuba County,15468
1833,Yuma County,32799
1834,Zapata County,1405
1835,Zavala County,2175


Now, we are re-merging this dataframe with ```df_zip_shp``` so that we can get the the number of children per county, the state each county is in, and the geometry for that county.

In [14]:
df = df_child_count.merge(df_zip_shp, on = ['county'], how = 'left')

Next, we use ```.drop_duplicates``` to drop all duplicate observations. We have the number of children per county for each zip code in each county, but we only need the number of children per county (without zip code breakdown). So, we drop all the duplicates based on the ```subset``` ```county``` and ```keep``` the first un-repeated observation for each county.

In [17]:
df = df.drop_duplicates(subset = 'county', keep = 'first')
df = df.drop(['zip'], axis = 1)
df

,county,child_num,state,geometry
0,Abbeville County,5203,SC,"POLYGON ((-82.62555 34.21630, -82.62667 34.217..."
5,Acadia Parish,19488,LA,"POLYGON ((-92.37568 30.36540, -92.37554 30.368..."
16,Accomack County,6908,VA,"POLYGON ((-75.69962 37.68312, -75.69764 37.684..."
56,Ada County,125475,ID,"POLYGON ((-116.51250 43.80716, -116.48385 43.8..."
96,Adair County,15569,KY,"POLYGON ((-85.45432 36.96768, -85.45391 36.969..."
...,...,...,...,...
41198,Yuba County,15479,CA,"POLYGON ((-121.51583 39.03013, -121.51504 39.0..."
41210,Yuma County,33177,CO,"POLYGON ((-102.61376 39.88982, -102.57632 39.8..."
41229,Zapata County,1405,TX,"POLYGON ((-99.45380 27.26506, -99.37184 27.318..."
41232,Zavala County,2175,TX,"POLYGON ((-99.74333 29.07505, -99.73887 29.076..."


### Dropping duplicates

To drop duplicate rows in a dataframe, you can specify the column in which duplicate rows are found as the parameter to ```subset``` in the ```drop_duplicates``` function. By specifying ```keep = 'first'```, the dataframe will keep the first instance of the duplicate row, but will drop any later occurrence of that row. If you would like to drop all instances of a row that appears more than once, you can specify ```keep = 'false'```.

In [ ]:
df_merged = df_merged.drop_duplicates(subset = 'zip', keep = 'first')
df_merged

We are now merging the completed merged dataframe with the dataframe with child counts so that we have not only the child counts by county, but also additional information such as the geometries so that we can use this dataframe in making later visualizations. In the code chunk below, we are specifying the two columns we want the two dataframes to merge on. This can be any number of columns that you choose as long as the corresponding columns in the two dataframes have matching names.

In [ ]:
df_final = df.merge(df_merged, on = ['zip', 'county'],how = 'left')
df_final

In [ ]:
df_final = df_final[['zip', 'county', 'state', 'has_child', 'child_num_y', 'age_code', 'geometry']]
df_final.rename(columns = {'child_num_y' : 'child_num'}, inplace = True)
df_final

### Convert the dataframe into a geodataframe

Convert the merged dataframe to a geodataframe so that it can export convert as a shapefile.

In [ ]:
gpd_df = gpd.GeoDataFrame(df_final)
gpd_df

,zip,county,state,has_child,child_num,age_code,geometry
0,18833,Bradford County,PA,0,319,K,"POLYGON ((-76.68205 41.60605, -76.68016 41.605..."
1,18079,Lehigh County,PA,0,76,G,"POLYGON ((-75.66384 40.74535, -75.65693 40.745..."
2,18350,Monroe County,PA,1,277,L,"POLYGON ((-75.52129 41.14508, -75.48068 41.135..."
3,23183,Gloucester County,VA,1,102,K,None
4,16652,Huntingdon County,PA,1,3715,I,"MULTIPOLYGON (((-77.93462 40.43937, -77.93272 ..."
...,...,...,...,...,...,...,...
37819,85023,Maricopa County,AZ,0,6312,K,"POLYGON ((-112.11629 33.62960, -112.11435 33.6..."
37820,76305,Wichita County,TX,0,1358,M,"POLYGON ((-98.60335 33.99502, -98.56224 33.994..."
37821,97369,Lincoln County,OR,0,27,I,"POLYGON ((-124.07285 44.77613, -124.07136 44.7..."
37822,98632,Cowlitz County,WA,0,11544,I,"POLYGON ((-123.23787 46.17862, -123.23636 46.1..."


### Export to shp file

Use the ```to_file``` command to export the given dataframe to the file path indicated in the parenthesis.

In [36]:
gpd_df.to_file('/hpc/group/codeplus22-vis/infousa_copy/children_count_by_county.shp')

In [2]:
df_test = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/children_count_by_county.shp')